In [1]:
from torchvision import transforms
import  torch.nn as nn
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
import numpy as np

In [2]:
## load the dataset 
from torchvision.datasets import CIFAR10
_tasks = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])
cifar = CIFAR10('data', train=True, download=True, transform=_tasks)


Files already downloaded and verified


In [3]:
## create training and validation split 
split = int(0.8 * len(cifar))
index_list = list(range(len(cifar)))
train_idx, valid_idx = index_list[:split], index_list[split:]

In [4]:
## create training and validation sampler objects
tr_sampler = SubsetRandomSampler(train_idx)
val_sampler = SubsetRandomSampler(valid_idx)

In [5]:
## create iterator objects for train and valid datasets
trainloader = DataLoader(cifar, batch_size=256, sampler=tr_sampler)
validloader = DataLoader(cifar, batch_size=256, sampler=val_sampler)

In [6]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        
        ## define the layers
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        self.conv3 = nn.Conv2d(32, 64, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.linear1 = nn.Linear(1024, 512)
        self.linear2 = nn.Linear(512, 10)
    
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(-1, 1024) ## reshaping 
        x = F.relu(self.linear1(x))
        x = self.linear2(x)
        return x

model = Model().cuda()

In [7]:
import torch.optim as optim
loss_function = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, weight_decay= 1e-6, momentum = 0.9, nesterov = True)

In [8]:
## run for 30 Epochs
for epoch in range(1, 14):
    train_loss, valid_loss = [], []
       ## training part 
    model.train()
    for data, target in trainloader:
        optimizer.zero_grad()
        data=data.to('cuda:0')
        target=target.to('cuda:0')
        output = model(data)
        loss = loss_function(output, target)
        loss.backward()
        optimizer.step()
        train_loss.append(loss.item()) 
    
    ## evaluation part 
    model.eval()
    for data, target in validloader:
        optimizer.zero_grad()
        data=data.to('cuda:0')
        target=target.to('cuda:0')
        output = model(data)
        loss = loss_function(output, target)
        valid_loss.append(loss.item())
    print('epoch:  ',epoch,'     train_loss : ',np.mean(train_loss),'     val_loss :',np.mean(valid_loss))
        ## dataloader for validation dataset 



epoch:   1      train_loss :  2.1322151665474958      val_loss : 1.900375697016716
epoch:   2      train_loss :  1.6867369177994456      val_loss : 1.6470004588365554
epoch:   3      train_loss :  1.4394961147551324      val_loss : 1.3830382406711579
epoch:   4      train_loss :  1.2953259755092061      val_loss : 1.3030912637710572
epoch:   5      train_loss :  1.1869424839687954      val_loss : 1.2087076902389526
epoch:   6      train_loss :  1.089129991971763      val_loss : 1.1513065248727798
epoch:   7      train_loss :  1.0109339739866317      val_loss : 1.0682095438241959
epoch:   8      train_loss :  0.9346284858740059      val_loss : 1.0304809927940368
epoch:   9      train_loss :  0.8700462135539693      val_loss : 1.0410577729344368
epoch:   10      train_loss :  0.8077853208134889      val_loss : 0.9359570801258087
epoch:   11      train_loss :  0.7479696729380614      val_loss : 0.93116664737463
epoch:   12      train_loss :  0.6906383588055897      val_loss : 0.9237694665

In [9]:
dataiter = iter(validloader)
data, labels = dataiter.next()
data = data.to('cuda:0')
output = model(data)

In [10]:
_, preds_tensor = torch.max(output, 1)
preds = np.squeeze(preds_tensor)

In [13]:
print ("Actual   :", labels[:10])
print ("Predicted:", preds[:10])


Actual   : tensor([8, 4, 5, 1, 3, 4, 5, 2, 2, 7])
Predicted: tensor([8, 5, 5, 1, 4, 4, 5, 8, 0, 5], device='cuda:0')


In [17]:
labels[:10]

tensor([8, 4, 5, 1, 3, 4, 5, 2, 2, 7])

In [30]:
import matplotlib.pyplot as plt

img=data.cpu().data.numpy()[0]